<a href="https://colab.research.google.com/github/ncolab/ViT-pytorch/blob/main/My_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvidia-smi

Sat Feb 26 20:28:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet34
import os

# level 0 : My dataset

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
cp /content/drive/MyDrive/thesis/MyDataset/classified/Nottingham_class/Nottingham_ph3.zip /content/

In [ ]:
!unzip Nottingham_ph3.zip

In [29]:
# Data
transform_train = transforms.Compose([
    transforms.RandomCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.ImageFolder(root='/content/Nottingham_ph3/train/',
                                transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=4, shuffle=True, num_workers=2)

testset = datasets.ImageFolder(root='/content/Nottingham_ph3/test/',
                                transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=True, num_workers=2)

In [30]:
batch = next(iter(trainloader))
batch[0].shape, batch[1].shape

(torch.Size([4, 3, 128, 128]), torch.Size([4]))

In [31]:
batch = next(iter(testloader))
batch[0].shape, batch[1].shape

(torch.Size([4, 3, 200, 169]), torch.Size([4]))

# level 1 : Initialization

In [2]:
!git clone https://github.com/ncolab/ViT-pytorch.git

Cloning into 'ViT-pytorch'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 170 (delta 32), reused 27 (delta 27), pack-reused 130
Receiving objects: 100% (170/170), 21.31 MiB | 33.56 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [6]:
cd ViT-pytorch

/content/ViT-pytorch


In [7]:
mkdir checkpoint

In [8]:
cd checkpoint

/content/ViT-pytorch/checkpoint


In [9]:
!wget https://storage.googleapis.com/vit_models/imagenet21k%2Bimagenet2012/ViT-B_16.npz

--2022-02-26 20:42:18--  https://storage.googleapis.com/vit_models/imagenet21k%2Bimagenet2012/ViT-B_16.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347502902 (331M) [application/octet-stream]
Saving to: ‘ViT-B_16.npz’

ViT-B_16.npz        100%[===================>] 331.40M   114MB/s    in 2.9s    

2022-02-26 20:42:21 (114 MB/s) - ‘ViT-B_16.npz’ saved [347502902/347502902]



In [10]:
cd ..

/content/ViT-pytorch


In [11]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 9054, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 9054 (delta 67), reused 43 (delta 26), pack-reused 8929
Receiving objects: 100% (9054/9054), 14.57 MiB | 23.83 MiB/s, done.
Resolving deltas: 100% (6180/6180), done.


In [12]:
cd apex

/content/ViT-pytorch/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [14]:
cd ..

/content/ViT-pytorch


In [ ]:
!pip install ml_collections

In [35]:
cd /content/ViT-pytorch

/content/ViT-pytorch


In [36]:
!python3 train.py --name MyDataset-1 --dataset MyDataset --model_type ViT-B_16 --pretrained_dir checkpoint/ViT-B_16.npz --fp16 --fp16_opt_level O2 --img_size 128 --train_batch_size 4 --eval_batch_size 4

Streaming output truncated to the last 5000 lines.
Validating... (loss=0.94370):  33% 23/69 [00:00<00:01, 41.32it/s]
Validating... (loss=2.18810):  33% 23/69 [00:00<00:01, 41.32it/s]
Validating... (loss=1.42036):  33% 23/69 [00:00<00:01, 41.32it/s]
Validating... (loss=1.42036):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.31882):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.20000):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.19398):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.11020):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.17792):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.04562):  42% 29/69 [00:00<00:00, 45.99it/s]
Validating... (loss=1.04562):  51% 35/69 [00:00<00:00, 48.74it/s]
Validating... (loss=1.24802):  51% 35/69 [00:00<00:00, 48.74it/s]
Validating... (loss=1.19375):  51% 35/69 [00:00<00:00, 48.74it/s]
Validating... (loss=1.22366):  51% 35/69 [00:00<00:00, 48.74it/s]
Validating... (loss=1.064

In [44]:
!git config --global user.email "n.colab.code@gmail.com"

In [38]:
!git add train.py

In [48]:
!git add utils/data_utils.py

In [45]:
!git add My_ViT.ipynb

fatal: pathspec 'My_ViT.ipynb' did not match any files


In [49]:
!git commit -m "Added My Dataset"

[main 2e1ff26] Added My Dataset
 1 file changed, 21 insertions(+), 2 deletions(-)


In [50]:
!git push -u origin master

error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/ncolab/ViT-pytorch.git'
